In [188]:
import random
import pickle as pkl
import argparse
import csv
import numpy as np
import scipy.stats
import time
from scipy.stats import chisquare
import pandas as pd

In [20]:
def load_data(ftrain, ftest):
    Xtrain, Ytrain, Xtest = [],[],[]
    with open(ftrain, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = [int(x) for x in row[0].split()]
            Xtrain.append(rw)

    with open(ftest, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = [int(x) for x in row[0].split()]
            Xtest.append(rw)

    ftrain_label = ftrain.split('.')[0] + '_label.csv'
    with open(ftrain_label, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = int(row[0])
            Ytrain.append(rw)
    #Remove this block before submission and remove Ytest
    Ytest = []
    ftest_label = ftest.split('.')[0] + '_label.csv'
    with open(ftest_label, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = int(row[0])
            Ytest.append(rw)

    print('Data Loading: done')
    return Xtrain, Ytrain, Xtest, Ytest

In [21]:
X_train, Y_train, X_test, Y_test = load_data('train.csv', 'test.csv')

Data Loading: done


In [74]:
x_train = np.array(X_train)
y_train = np.array(Y_train)
x_test = np.array(X_test)
y_test = np.array(Y_test)


In [103]:
np.bincount(np.array([1,2,3,4,5,6,7,8,23]))
y_train = np.reshape(y_train,(40000,1))

In [107]:
x_train = np.append(x_train, y_train, axis = 1)

In [108]:
np.shape(x_train)

(40000, 275)

In [31]:
class TreeNode():
    def __init__(self, data='T',children=[-1]*5):
        self.nodes = list(children)
        self.data = data

    def save_tree(self,filename):
        obj = open(filename,'w')
        pkl.dump(self,obj)

In [151]:
def info_gain(Tdata):
    max_gain = 0
    max_gain_col = ""
    counts = np.bincount(Tdata.T[-1])
    sys_ent = scipy.stats.entropy([x/np.shape(Tdata.T[-1])[0] for x in counts], base = 2)
    for col in range(np.shape(Tdata)[1] - 1):
        valDict = {}
        for row in range(np.shape(Tdata)[0]): 
            if str(Tdata[row,col]) in valDict:
                if Tdata[row,-1] == 1:
                    valDict[str(Tdata[row,col])][0] += 1
                else:
                    valDict[str(Tdata[row,col])][1] += 1
            else:
                if Tdata[row,-1] == 1:
                    valDict[str(Tdata[row,col])] = [1,0]
                else:
                    valDict[str(Tdata[row,col])] = [0,1]
        col_counts = np.bincount(Tdata[:,col])
        ent = 0
        for value in valDict:
            ent += col_counts[int(value)]/np.sum(col_counts) * scipy.stats.entropy([x/np.sum(valDict[value]) for x in valDict[value]], base = 2)
        col_gain = sys_ent - ent
        if col_gain > max_gain:
            max_gain = col_gain
            max_gain_col = col
    return max_gain_col

In [152]:
A= time.time()
info_gain(x_train)
print(time.time() - A)

39.216637134552


In [184]:
a = np.array([1,1,1,1,1])
b = chisquare(x_train[1])